# CMEMS OSTIA Marine Heatwave Time Series Plotter

    Version: 2.0
    Date:    17/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory ) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

    Version: 2.0
    Date:    17/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory ) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

**What specific tools does this notebook use?**


***

Python is divided into a series of modules that each contain a series of methods for specific tasks. The box below imports all of the moduls we need to complete our plotting task

In [ ]:
%matplotlib inline

# standard tools
import os
import sys
import json
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import glob
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# where the data should have been downloaded to:
download_dir_path = "/home/jovyan/work/products"

# subset image: cut a relevant section out of an image for area averaging. If false, whole area will be used.
# THERE IS NO AREA WEIGHTING IN THE AVERAGING HERE! Subset_extents [lon1,lon2,lat1,lat2] describes the section.
subset_image = True
subset_extents = [-165.0, -145.0, 10.0, 30.0] # +/- 10 degree box around Hawaii
show_plots = False

Find all of our data files. We do this by concatenating two lists, as there are two distinct file types.

In [ ]:
SST_files = []
my_files = glob.glob(download_dir_path+'/METOFFICE-GLO-SST-L4-RAN*')
for SST_file in sorted(my_files):
    SST_files.append(SST_file)
    
my_files = glob.glob(download_dir_path+'/METOFFICE-GLO-SST-L4-NRT*')
for SST_file in sorted(my_files):
    SST_files.append(SST_file)

Now compare the target SST against our reference (e.g. the climatology we downloaded in the CMEMS_OSTIA_Climatology_Data_Downloader.ipynb notebook)

In [ ]:
#load the variables we want from our target SST
ds = xr.open_dataset(SST_files[-1])
lat = ds.lat.data
lon = ds.lon.data
time = ds.time.data
SST = ds.analysed_sst.data
if np.nanmean(SST) > 100:
    SST = SST - 273.15
ds.close()

if subset_image:
    ii = np.where((lon >= subset_extents[0]) & (lon <= subset_extents[1]))[0]
    jj = np.where((lat >= subset_extents[2]) & (lat <= subset_extents[3]))[0]
    SST = SST[:,jj[0]:jj[-1],ii[0]:ii[-1]]
    lon = lon[ii]
    lat = lat[jj]
    
SST_area_average = np.nanmin(np.nanmin(SST,axis=1),axis=1)

# set up a dummy file
clim_SST = np.zeros(np.shape(SST))

# now we get the average data
for SST_file in SST_files:
    print(SST_file)
    ds = xr.open_dataset(SST_file)
    iter_SST = ds.analysed_sst.data
    ds.close()
    
    if np.nanmean(iter_SST) > 100:
        iter_SST = iter_SST - 273.15
    
    if subset_image:
        iter_SST = iter_SST[:,jj[0]:jj[-1],ii[0]:ii[-1]]
    
    if show_plots:
        plt.pcolormesh(lon, lat, np.squeeze(np.nanmean(iter_SST, axis=0)), \
                       vmin=np.nanmin(SST), vmax=np.nanmax(SST), cmap=plt.cm.Spectral_r)
        plt.colorbar()
        plt.show()
    clim_SST = clim_SST + iter_SST

# rescale
clim_SST = clim_SST/len(SST_files)
clim_SST_area_average = np.nanmin(np.nanmin(clim_SST,axis=1),axis=1)

Plot the spatial anomaly

In [ ]:
vmax = np.nanmax(abs(SST - clim_SST))
for SST_file in SST_files:
    print(SST_file)
    ds = xr.open_dataset(SST_file)
    iter_SST = ds.analysed_sst.data
    ds.close()
    
    if np.nanmean(iter_SST) > 100:
        iter_SST = iter_SST - 273.15
    
    if subset_image:
        iter_SST = iter_SST[:,jj[0]:jj[-1],ii[0]:ii[-1]]    
    
    plt.pcolormesh(lon, lat, np.squeeze(np.nanmean(iter_SST - clim_SST, axis=0)),vmin=vmax*-1,vmax=vmax,cmap=plt.cm.RdBu_r)
    plt.colorbar()
    plt.show()

In [ ]:
plt.plot(SST_area_average,'r')
plt.plot(clim_SST_area_average,'k--')

make climate stripes

In [ ]:
# make matrix
stripe_array = np.ones([20,len(SST_files)])*np.nan

count = -1
for SST_file in SST_files:
    count = count + 1
    print(SST_file)
    ds = xr.open_dataset(SST_file)
    iter_SST = ds.analysed_sst.data
    ds.close()
    
    if np.nanmean(iter_SST) > 100:
        iter_SST = iter_SST - 273.15
    
    if subset_image:
        iter_SST = iter_SST[:,jj[0]:jj[-1],ii[0]:ii[-1]]
    
    iter_SST_anom = iter_SST - clim_SST
    stripe_array[:, count] = np.nanmean(iter_SST_anom)

plt.pcolormesh(stripe_array,vmin=vmax*-1,vmax=vmax,cmap=plt.cm.RdBu_r)
plt.colorbar()